In [1]:
import pandas as pd
import ast
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

df = pd.read_csv('/home/sasidharreddy/study/OELP/data.csv', delimiter=',')
df_key=pd.read_csv('/home/sasidharreddy/study/OELP/keywords.csv', delimiter='\t')
name=[]
genres=[]
story=[]
rating=[]
directors=[]
pop_of_directors=[]
cast=[]
pop_of_cast=[]
for i in df['Name']:
    mve_list = ast.literal_eval(i)
    name.append(mve_list[0])
df['story'] = df['story'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stopwords.words('english')]))
for i in df['genres']:
    genres_list = ast.literal_eval(i)
    genres.append(genres_list)
for i in df['story']:
    storyline=ast.literal_eval(i)
    story.append(storyline[0].split())
for i in df['rating']:
    rating_list=ast.literal_eval(i)
    rating.append(rating_list)

d1={}
cas,rank1=[],[]
for i in df['cast']:
    i=ast.literal_eval(i)
    for j in i:
        cas.append(j)
for i in df['popularity of cast']:
    i=ast.literal_eval(i)
    for j in i:
        if j is None:
            j=5000
        rank1.append(j)
for i in range(len(cas)):
    d1[cas[i]] = rank1[i]

C = []                             # M is the matrix where each row represents each movie and column represents each unique keyword
for i in df['cast']:
    x = eval(i)
    l = []
    for j in d1.keys():
        if(j in x):
            if(int(d1[j])==0):
                l.append(0.09)
            else:
                l.append(1-(int(d1[j])/11000))
        else:
            l.append(0)
    C.append(l)

d={}
dir,rank=[],[]
for i in df['directors']:
    i=ast.literal_eval(i)
    for j in i:
        dir.append(j)
for i in df['popularity of director']:
    i=ast.literal_eval(i)
    for j in i:
        rank.append(j)
for i in range(len(dir)):
    d[dir[i]] = rank[i]

R = []                             # M is the matrix where each row represents each movie and column represents each unique keyword
for i in df['directors']:
    x = eval(i)
    l = []
    for j in d.keys():
        if(j in x):
            if(int(d[j])==0):
                l.append(0.09)
            else:
                l.append(1-(int(d[j])/11000))
        else:
            l.append(0)
    R.append(l)
s = set()                           # set of all unique keywords
for i in df_key['k_h']:
    x = eval(i)
    for j in x.keys():
        s.add(j)                             
M = []                             # M is the matrix where each row represents each movie and column represents each unique keyword
for i in df_key['k_h']:
    x = eval(i)
    l = []
    for j in s:
        if(j in x.keys()):
            if(x[j]==0):
                l.append(0.5)
            else:
                l.append(int(x[j]))
        else:
            l.append(0)
    M.append(l)

d2={}
writ,rank2=[],[]
for i in df['writers']:
    i=ast.literal_eval(i)
    for j in i:
        writ.append(j)
for i in df['popularity of writer']:
    i=ast.literal_eval(i)
    for j in i:
        if j is None:
            j=5000
        rank2.append(j)
for i in range(len(writ)):
    d2[writ[i]] = rank2[i]

W = []                             # M is the matrix where each row represents each movie and column represents each unique keyword
for i in df['writers']:
    x = eval(i)
    l = []
    for j in d2.keys():
        if(j in x):
            if(int(d2[j])==0):
                l.append(0.09)
            else:
                l.append(1-(int(d2[j])/11000))
        else:
            l.append(0)
    W.append(l)

unique_genres = np.unique([genre for sublist in genres for genre in sublist])
unique_story=np.unique([sto for sublist in story for sto in sublist])

genre_matrix = [[1 if word in sublist else 0 for word in unique_genres] for sublist in genres]
key_word_matrix = np.array(M)
cast_matrix=np.array(C)
director_matrix=np.array(R)
writer_matrix=np.array(W)
story_matrix=[[1 if word in sublist else 0 for word in unique_story] for sublist in story]
print(len(genre_matrix[0]))
print(len(key_word_matrix[0]))
print(len(cast_matrix[0]))
print(len(director_matrix[0]))
print(len(writer_matrix[0]))
print(len(story_matrix[0]))
#normalizing the matrices
def normalize_matrix(matrix, new_min, new_max):
    min_val = np.min(matrix)
    max_val = np.max(matrix)
    matrix_range = max_val - min_val

    normalized_matrix = (matrix - min_val) / matrix_range * (new_max - new_min) + new_min
    return normalized_matrix


genre_normalized_matrix = normalize_matrix(genre_matrix, 0, 1)
key_words_normalized_matrix = normalize_matrix(key_word_matrix, 0, 1)
cast_genre_normalized_matrix = normalize_matrix(cast_matrix, 0, 1)
director_normalized_matrix = normalize_matrix(director_matrix, 0, 1)
writer_normalized_matrix = normalize_matrix(writer_matrix, 0, 1)
story_normalized_matrix = normalize_matrix(story_matrix, 0, 1)

final_matrix=np.hstack((genre_normalized_matrix,key_words_normalized_matrix,cast_genre_normalized_matrix,director_normalized_matrix,writer_normalized_matrix,story_normalized_matrix))
print(len(final_matrix[0]))

from sklearn.decomposition import NMF
# Sample movie-feature matrix (replace this with your actual data)
movie_feature_matrix = np.array(final_matrix)

def nmf_reduction(M, k):
    model = NMF(n_components=k)
    W = model.fit_transform(M)
    H = model.components_
    return W,H
w,h = nmf_reduction(movie_feature_matrix,44)
reconstructed_matrix = np.dot(w,h)
print(w)
print(h)
cos_sim = cosine_similarity(reconstructed_matrix)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sasidharreddy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


20
22827
16973
1160
2250
9654
52884
[[0.         0.         0.00202055 ... 0.         0.         0.00873257]
 [0.06357881 0.         0.         ... 0.         0.30333325 0.02145271]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.06972398 0.         0.         ... 0.01738057 0.         0.        ]
 [0.         0.19868768 0.         ... 0.         0.         0.01494043]
 [0.         0.         0.         ... 0.         0.         0.        ]]
[[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  1.89706603e-02 0.00000000e+00]
 [5.15472588e+00 1.83858374e-01 0.00000000e+00 ... 9.32464351e-04
  1.72935368e-02 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  1.69561966e-02 0.00000000e+00]
 ...
 [0.00000000e+00 3.35856136e-02 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.89414769e-02 1.4

In [14]:
movie_name = 'The Nun II'
try:
    index = name.index(movie_name)
    print(f"The input movie is {movie_name}")
except ValueError:
    print(f"{movie_name} is not present in the list.")

import numpy as np
# Assuming you have an array named 'your_array'
max_array=np.array(cos_sim[index])

# Get the indices of the 11 maximum values
max_indices = np.argsort(max_array)[:-22:-1]
max_indices=max_indices[1:]
# Print the indices

inde=0
for i in max_indices:
    inde=inde+1
    print(f'{inde}-{name[i]}')

The input movie is The Nun II
1-Jeepers Creepers Reborn
2-Annabelle Comes Home
3-The Curse of the Weeping Woman
4-The Visit
5-Influencer
6-Slender Man
7-Sick
8-The Conjuring 2
9-The Conjuring 3
10-Truth or Dare
11-Eight for Silver
12-Us
13-In the Earth
14-The Night House
15-The Black Phone
16-The Ritual
17-There's Someone Inside Your House
18-Smile
19-Annabelle: Creation
20-Nefarious
